In [15]:
import json
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.docstore.document import Document

embeddings = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

In [52]:
i = 24
docs = []
sents = []

with open(f"/home/ullriher/aic_averitec/data_store/output_dev/{i}.json") as f:
    for line in f:
        data = json.loads(line)
        docs.append("\n".join(data["url2text"]))
        sents.extend(data["url2text"])
        
with open("data/dev.json") as f:
    claims = json.load(f)
    claim = claims[i]["claim"]
claim, docs[:i]

('Germany’s Foreign Minister Heiko Maas said that Thailand’s King Maha Vajiralongkorn didn’t do anything illegal while at his German residence.',
 ['Berlin: Germany\'s Foreign Minister says Thailand\'s King Maha Vajiralongkorn should not be conducting politics from the European country, where he spends much of his time.\nHeiko Maas responded to a question by a Greens MP at the Bundestag at a time when King Vajiralongkorn\'s monarchy faces unprecedented calls for reform from protesters in Thailand.\n"We have made it clear that politics concerning Thailand should not be conducted from German soil," Maas told parliament.\n"If there are guests in our country that conduct their state business from our soil we would always want to act to counteract that."\nGreens parliamentarian Frithjof Schmidt had asked "Why does the German government tolerate this extremely unusual - and in my opinion illegal - behaviour of a foreign head of state conducting politics on German soil?"\nThousands of protest

In [37]:
# load .env
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["SERPAPI_KEY"]

'38cce24db1e96f8b137712dc1b249929ff12f8ba6033b2c0b7d05fd796074915'

In [39]:
import serpapi
serpapi.search(q=claim, engine="google",api_key=os.environ["SERPAPI_KEY"])

{
    "search_metadata": {
        "id": "6683fa0734ff9550b231bf33",
        "status": "Success",
        "json_endpoint": "https://serpapi.com/searches/eac03ae165c4813c/6683fa0734ff9550b231bf33.json",
        "created_at": "2024-07-02 13:00:55 UTC",
        "processed_at": "2024-07-02 13:00:55 UTC",
        "google_url": "https://www.google.com/search?q=Republican+Matt+Gaetz+was+part+of+a+company+that+had+to+pay+75+million+in+hospice+fraud.+They+stole+from+dying+people.&oq=Republican+Matt+Gaetz+was+part+of+a+company+that+had+to+pay+75+million+in+hospice+fraud.+They+stole+from+dying+people.&sourceid=chrome&ie=UTF-8",
        "raw_html_file": "https://serpapi.com/searches/eac03ae165c4813c/6683fa0734ff9550b231bf33.html",
        "total_time_taken": 2.93
    },
    "search_parameters": {
        "engine": "google",
        "q": "Republican Matt Gaetz was part of a company that had to pay 75 million in hospice fraud. They stole from dying people.",
        "google_domain": "google.com",
  

In [17]:
len(docs)

1064

In [27]:
Chroma().delete_collection()
db = Chroma.from_documents([Document(s) for s in docs], embeddings)

In [42]:
claim

'Republican Matt Gaetz was part of a company that had to pay 75 million in hospice fraud. They stole from dying people.'

In [41]:
db.similarity_search(claim, k=7)

[Document(page_content='In recent months, a national writer for Mother Jones magazine named Stephanie Mencimer has been digging into a story about the local roots of FloridaMan and U.S. Congressman Matt Gaetz.\nIt finally dropped last week. And it should be mandatory reading for residents of the Panhandle who need to understand just how badly our community has been conned by small and corrupt confederacy political players who created Matt Gaetz.\nHere are just a few highlights from Mencimer’s work:\n► While Matt rants against boogeymen like "socialism" and "leftist" healthcare hijackers, the Gaetz family money came from a hospice company called "Vitas" that profited from Medicare and Medicaid payments.\nThe company made the Gaetz family rich when they sold it. And later, it was sued by the Justice Department for allegedly defrauding Medicare, as Mencimer explains:\n“In 2004, Don Gaetz and his partners cashed in, selling the hospice company to the parent company of the plumbing behemoth

In [32]:
docs = db.similarity_search(claim)
print(claim,"\n---")
print(docs[0].page_content)
print(docs[1].page_content)
print(docs[2].page_content)

Republican Matt Gaetz was part of a company that had to pay 75 million in hospice fraud. They stole from dying people. 
---
In recent months, a national writer for Mother Jones magazine named Stephanie Mencimer has been digging into a story about the local roots of FloridaMan and U.S. Congressman Matt Gaetz.
It finally dropped last week. And it should be mandatory reading for residents of the Panhandle who need to understand just how badly our community has been conned by small and corrupt confederacy political players who created Matt Gaetz.
Here are just a few highlights from Mencimer’s work:
► While Matt rants against boogeymen like "socialism" and "leftist" healthcare hijackers, the Gaetz family money came from a hospice company called "Vitas" that profited from Medicare and Medicaid payments.
The company made the Gaetz family rich when they sold it. And later, it was sued by the Justice Department for allegedly defrauding Medicare, as Mencimer explains:
“In 2004, Don Gaetz and his

In [24]:
results = model.rank(claim, sents, return_documents=True, top_k=40)


In [25]:
results

[{'corpus_id': 144,
  'score': 0.70080024,
  'text': 'A Facebook post said, "Matt Gaetz has had numerous DUIs and his dad got him off each time. He was part of a company that had to pay 75 million in hospice fraud.'},
 {'corpus_id': 193,
  'score': 0.70080024,
  'text': 'A Facebook post said, "Matt Gaetz has had numerous DUIs and his dad got him off each time. He was part of a company that had to pay 75 million in hospice fraud.'},
 {'corpus_id': 682,
  'score': 0.62565136,
  'text': 'TALLAHASSEE — The U.S. Department of Justice has sued the hospice company founded by Florida Senate President Don Gaetz, accusing the company of engaging in Medicare fraud for more than 11 years, including during the time Gaetz was vice chairman.'}]

In [54]:
from langchain_community.retrievers import KNNRetriever

ret = KNNRetriever.from_texts(
    docs, embeddings, k=5
)
ret.get_relevant_documents(claim)

[Document(page_content="BERLIN (Reuters) - Germany's foreign minister has said that Thailand's King Maha Vajiralongkorn should not be conducting politics from the European country, where he spends much of his time.\nForeign Minister Heiko Maas responded to a question by a Greens member of parliament in the Bundestag on Wednesday at a time that King Vajiralongkorn's monarchy faces unprecedented calls for reform from protesters in Thailand.\nFollow us on our official WhatsApp channel for breaking news alerts and key updates!"),
 Document(page_content='The German Foreign Minister Heiko Maas is against Thailand’s King Maha Vajiralongkorn (or Rama X) ruling from Bavaria where he stays with his harem in a hotel.\nThe government official was asked by a Greens MP in the Bundestag about the King conducting politics in Germany instead of in his own country.\n“Why does the German government tolerate this extremely unusual – and in my opinion illegal – behaviour of a foreign head of state conducti

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)

In [22]:
from sentence_transformers import CrossEncoder

# Load the model, here we use our base sized model
model = CrossEncoder("mixedbread-ai/mxbai-rerank-xsmall-v1")


# Example query and documents
query = "Who wrote 'To Kill a Mockingbird'?"
documents = [
    "'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature.",
    "The novel 'Moby-Dick' was written by Herman Melville and first published in 1851. It is considered a masterpiece of American literature and deals with complex themes of obsession, revenge, and the conflict between good and evil.",
    "Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961.",
    "Jane Austen was an English novelist known primarily for her six major novels, which interpret, critique and comment upon the British landed gentry at the end of the 18th century.",
    "The 'Harry Potter' series, which consists of seven fantasy novels written by British author J.K. Rowling, is among the most popular and critically acclaimed books of the modern era.",
    "'The Great Gatsby', a novel written by American author F. Scott Fitzgerald, was published in 1925. The story is set in the Jazz Age and follows the life of millionaire Jay Gatsby and his pursuit of Daisy Buchanan."
]

# Lets get the scores
results = model.rank(query, documents, return_documents=True, top_k=3)


config.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/142M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]